In [1]:
from pynq import PL
import xrfdc
from pynq import Overlay

PL.reset()
ol = Overlay('./20241208_final.bit')
dma = ol.axi_dma_0
rf = ol.usp_rf_data_converter_0

In [2]:
import numpy as np
from pynq import allocate
import plotly.graph_objects as go
import plotly.express as px

In [11]:
n = 52 * 8
curr_data = np.zeros((129, 52), dtype=np.complex_)
curr_data[0] = np.ones(52) * 5
out_buffer = allocate(n, dtype=np.int32)

# Create the live plot
fig = go.FigureWidget()
fig_imshow = px.imshow(curr_data.real, origin='lower')
fig.add_trace(go.Heatmap(fig_imshow.data[0], name='csi'))
fig.update_layout(width=750, height=750)
fig

FigureWidget({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>c…

In [12]:
adc_tile = rf.adc_tiles[2]
adc_block = adc_tile.blocks[0]

# Sampling frequency (MHz)
fs = 122.88
# Mixer frequency (MHz)
fm = 2412  # WiFi channel 1
adc_block.Dither = 1
adc_block.MixerSettings['Freq']= -fm # Negative because we're in Nyquist zone 1
adc_block.UpdateEvent(xrfdc.EVENT_MIXER)

In [ ]:
while True:
    # Receive the CSI
    dma.recvchannel.transfer(out_buffer)
    dma.recvchannel.wait()
    real = []
    imag = []
    for i in range(n):
        val = out_buffer[i] & 0xFFFF
        if val >= 32768:
            imag.append(np.int32(0xFFFF0000 | val))
        else:
            imag.append(val)
        real.append((out_buffer[i]>>16))
    csi = (np.array(real) + 1j*np.array(imag)).reshape((8, 52))
    # Update the buffer
    curr_data[9:] = curr_data[1:-8]
    curr_data[1:9] = csi
    # Update the plot
    fig.update_traces(
        selector={'name': 'csi'},
        z=np.log(np.abs(curr_data) + 1)
    )

In [52]:
data = None

while data is None or len(data) < 1024:
    # Receive the CSI
    dma.recvchannel.transfer(out_buffer)
    dma.recvchannel.wait()
    real = []
    imag = []
    for i in range(n):
        val = out_buffer[i] & 0xFFFF
        if val >= 32768:
            imag.append(np.int32(0xFFFF0000 | val))
        else:
            imag.append(val)
        real.append((out_buffer[i]>>16))
    csi = (np.array(real) + 1j*np.array(imag)).reshape((8, 52))
    # Update the buffer
    if data is None:
        data = csi
    else:
        data = np.append(data, csi, axis=0)

In [53]:
data.shape

(1024, 52)

In [55]:
import pandas as pd

In [60]:
df = pd.DataFrame(np.abs(data))

In [62]:
model = sm.tsa.MarkovRegression(df, k_regimes=2)

ValueError: Must have univariate endogenous data.